# Week 7 — Part 03: Error handling that teaches the user what to do

**Estimated time:** 60–90 minutes

## Learning Objectives

- Write error messages that explain what went wrong and what to do next
- Classify common failure types (input, config, external, parsing)
- Implement helper functions that validate inputs early
- Keep user-facing errors short and actionable


## Overview

Good error messages reduce support burden.

A good error message contains:

- what went wrong
- where it happened
- what to try next

---

## Underlying theory: errors are part of your product surface

When something fails, the error message becomes the user experience.

You can think of your program as an interface that returns either:

- a result
- or an explainable failure

Practical failure taxonomy (Level 1):

- **input errors** (missing/empty file, missing columns)
- **config errors** (missing API key)
- **external errors** (timeouts, rate limits)
- **output/parse errors** (invalid JSON from model)

Good errors point to the next action instead of just crashing.

In [ ]:
from pathlib import Path


def require_file(path: str) -> Path:
    p = Path(path).expanduser()
    if not p.exists():
        raise FileNotFoundError(
            f"Input file not found: {p}. Check the path or run --help for usage."
        )
    if p.stat().st_size == 0:
        raise ValueError(f"Input file is empty: {p}. Provide a non-empty CSV.")
    return p


def require_columns(df, required: list[str]) -> None:
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(
            f"Missing required columns: {missing}. Update your schema or fix the input file."
        )


print("error helpers ready")

In [ ]:
def classify_error(exc: Exception) -> str:
    # TODO: categorize errors into input/config/external/output types.
    raise NotImplementedError


def user_facing_message(exc: Exception) -> str:
    # TODO: map error type to a short, actionable message.
    raise NotImplementedError


print("Implement classify_error() and user_facing_message().")

## Self-check

- If a beginner runs your project wrong, does the error message teach them what to do?

## References

- Python errors/exceptions: https://docs.python.org/3/tutorial/errors.html